In [1]:
!pip install matplotlib

In [27]:
%%file gridding.py

from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt
import sys

# Function to transform coordinates u and v to wavelengths
def transform_coordinates(data):
    c = 299792458.0  # Speed of light in meters per second
    data[:, 0] = data[:, 0] * data[:, 6] / c
    data[:, 1] = data[:, 1] * data[:, 6] / c
    return data

# Function to read the input file
def read_data(input_file):
    data = []
    with open(input_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            values = line.strip().split(',')
            data.append([float(value) for value in values])
    return np.array(data)

# Function to perform the gridding process
def grid_data(local_data, N, delta_x_rad):
    local_F_r = np.zeros((N, N), dtype=np.float64)
    local_F_i = np.zeros((N, N), dtype=np.float64)
    local_W_t = np.zeros((N, N), dtype=np.float64)

    weight = local_data[:, 5]
    ik = np.round(local_data[:, 0]/ delta_x_rad).astype(int) + N // 2
    jk = np.round(local_data[:, 1]/ delta_x_rad).astype(int) + N // 2

    valid_indices = (ik >= 0) & (ik < N) & (jk >= 0) & (jk < N)

    local_F_r[ik[valid_indices], jk[valid_indices]] += weight[valid_indices] * local_data[valid_indices, 3]
    local_F_i[ik[valid_indices], jk[valid_indices]] += weight[valid_indices] * local_data[valid_indices, 4]
    local_W_t[ik[valid_indices], jk[valid_indices]] += weight[valid_indices]

    return local_F_r, local_F_i



# Parse command-line arguments
if len(sys.argv) != 7:
    print("Usage: mpirun -n num_processes python gridding.py -i data_file -d deltax -N image_size")
    sys.exit(1)

if sys.argv[1] != '-i':
    print("Error: You must specify the data file using -i")
    sys.exit(1)

if sys.argv[3] != '-d':
    print("Error: You must specify the value of deltax using -d")
    sys.exit(1)

if sys.argv[5] != '-N':
    print("Error: You must specify the image size using -N")
    sys.exit(1)


input_file = sys.argv[2]
delta_x_arcsec = float(sys.argv[4])
delta_x_rad = (np.pi / 180 / 3600) * delta_x_arcsec
N = int(sys.argv[6])
    
# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = read_data(input_file)
    data = transform_coordinates(data)
    data_shape = data.shape

else:
    data = None
    data_shape = None
    
# Broadcast the shape of data from root process to all processes
data_shape = comm.bcast(data_shape, root=0)
    
# Scatter the data from the root process to all processes
recv_data = np.empty((data_shape[0] // size, data_shape[1]), dtype = np.float64)
comm.Scatter(data, recv_data, root = 0)

# Perform the gridding process on all processes
local_F_r, local_F_i = grid_data(recv_data, N, delta_x_rad)

# Espera a que todos los procesos terminen
comm.Barrier()

# Crear arrays vacíos para recopilar las matrices M_real, M_img y M_peso en el proceso raíz
if rank == 0:
    gathered_M_real = np.empty((size, N, N), dtype = np.float64)
    gathered_M_imaginario = np.empty((size, N, N), dtype = np.float64)
else:
    gathered_M_real = None
    gathered_M_imaginario = None

# Gather the results from all processes back to the root process
comm.Gather(local_F_r, gathered_M_real, root=0)
comm.Gather(local_F_i, gathered_M_imaginario, root=0)

# The root process (rank 0) join partials results
if rank == 0:
    m_r = np.zeros((N, N))
    m_i = np.zeros((N, N))
    
    for matrix in gathered_M_real:
        m_r += matrix
        
    for matrix in gathered_M_imaginario:
        m_i += matrix
    
    grid = m_r + 1j * m_i
    dirty_image = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(grid)))

    plt.imshow(np.abs(dirty_image), origin='lower', extent=[0, N, 0, N])
    plt.colorbar()
    plt.show()
    plt.savefig('dirtyImage.png')
    

# Finaliza MPI
MPI.Finalize()

Overwriting gridding.py


In [28]:
!mpirun -n 8 python gridding.py -i hltau_completo_uv.csv -d 0.1 -N 256

Figure(640x480)
